# Análisis Consolidado del Presupuesto Vigente Multianual por Objeto de Gasto y Año

Este notebook realiza un análisis detallado y automatizado del presupuesto vigente del sector público, utilizando datos históricos de varios años. El objetivo principal es obtener, para cada año y objeto de gasto, el monto vigente correspondiente a la fecha de corte más reciente publicada para el mes de diciembre, permitiendo así una visión consolidada y comparable del cierre presupuestario anual.

**¿Qué hace este notebook?**
- Carga y prepara los datos presupuestarios multianuales desde un archivo CSV.
- Filtra los registros para los años de interés (2019-2024) y el mes de diciembre, asegurando que se utilicen únicamente los datos de cierre anual.
- Identifica automáticamente la fecha de corte más reciente para cada año, garantizando que el análisis se base en la información más actualizada disponible.
- Calcula el monto vigente total por año y por objeto de gasto, así como los totales generales consolidados.
- Genera reportes tabulares que facilitan la comparación interanual y la conciliación con cifras oficiales.

**¿Para qué sirve?**
- Permite auditar y comparar la evolución del presupuesto vigente a lo largo de los años y entre diferentes conceptos de gasto.
- Facilita el control de gestión y la conciliación con reportes oficiales publicados por organismos de control.
- Proporciona una base sólida para análisis de tendencias, elaboración de informes y toma de decisiones en materia presupuestaria.

**Estructura del notebook:**
1. Carga y preparación de datos.
2. Filtrado por años y mes de interés.
3. Identificación de la fecha de corte máxima por año.
4. Cálculo de montos vigentes por objeto de gasto y año.
5. Reportes consolidados y totales generales.

Este enfoque garantiza transparencia, reproducibilidad y precisión en el análisis del presupuesto vigente multianual.

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os

# Cargar variables de entorno desde .env
load_dotenv()

csv_path = os.getenv("csv_path")
# Actualizar el nombre del archivo si es necesario

# Leer el archivo CSV en un DataFrame
# Asegúrate de que el archivo contiene las columnas: anio, objetoGasto, montoVigente, fechaCorte, mes
# Si el archivo ya está cargado como df_all_years, omite esta celda

df_all_years = pd.read_csv(csv_path, encoding="utf-8", sep=",")

In [ ]:
# Filtrar para los años de interés y mes 12 (diciembre)
df_vigente = df_all_years[(df_all_years['anio'].between(2019, 2024)) & (df_all_years['mes'] == 12)].copy()
df_vigente.head()

### Paso 1: Obtener la fechaCorte máxima por año (para diciembre)

In [ ]:
# Para cada año, obtener la fechaCorte máxima (última actualización de diciembre)
df_fechas_max = df_vigente.groupby(['anio'], as_index=False)['fechaCorte'].max()
df_fechas_max.head()

### Paso 2: Filtrar para quedarnos solo con los registros de la fechaCorte máxima de cada año

In [ ]:
# Unir para obtener todos los registros de la fechaCorte máxima de cada año
df_max_fecha = pd.merge(df_vigente, df_fechas_max, on=['anio', 'fechaCorte'])
df_max_fecha = df_max_fecha[['anio','objetoGasto','montoVigente','fechaCorte','mes']]
df_max_fecha = df_max_fecha.sort_values(['anio','objetoGasto'])
df_max_fecha.head()

### Paso 3: Sumar montoVigente por año y objetoGasto, y mostrar el total general

In [ ]:
# Sumar montoVigente por año y objetoGasto
reporte_vigente = df_max_fecha.groupby(['anio','objetoGasto'], as_index=False)['montoVigente'].sum().sort_values(['anio','objetoGasto'])
reporte_vigente

In [ ]:
# Sumar el total general
suma_total_vigente = df_max_fecha['montoVigente'].sum()
print(f"TOTAL GENERAL montoVigente (todos los años): {suma_total_vigente:,.2f}")

### Paso 4: Reporte anual consolidado (sin agrupar por objetoGasto)

En este paso, se genera un reporte que agrupa únicamente por año, sumando el montoVigente total de todos los objetos de gasto para cada año. Esto permite visualizar el presupuesto vigente consolidado anual y facilita la comparación global entre años.

In [ ]:
# Agrupar solo por año, sumando montoVigente de todos los objetos de gasto
reporte_anual_vigente = df_max_fecha.groupby(['anio'], as_index=False)['montoVigente'].sum()
reporte_anual_vigente = reporte_anual_vigente.sort_values(['anio'])
reporte_anual_vigente

In [ ]:
# Mostrar el total general de montoVigente para el reporte anual consolidado
suma_total_anual_vigente = reporte_anual_vigente['montoVigente'].sum()
print(f"TOTAL GENERAL montoVigente (reporte anual consolidado): {suma_total_anual_vigente:,.2f}")